In [2]:
from heapq import heappop, heappush

def calc_manhattan_distance(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    return(abs(x1-x2)+abs(y1-y2))

class Maze():
    def __init__(self, maze_grid):
        self.maze_grid = maze_grid
        self.maze_list = self.maze_unpack(maze_grid)
        self.start = [0,0]
        self.length = len(self.maze_list)
        self.finish = [self.length-1,self.length-1]
        
    def get_adjacent_cells(self, coords):
        maze = self.maze_list
        x,y = coords
        maze_l = self.length
        
        adj_cells = []
        #check right
        if x + 1 < maze_l and maze[y][x+1] == '.': adj_cells.append([x+1,y])
        #check left
        if x - 1 >= 0 and maze[y][x-1] == '.':adj_cells.append([x-1,y])
        #check down
        if y + 1 < maze_l and maze[y+1][x] == '.':adj_cells.append([x,y+1])
        #check up
        if y - 1 >= 0 and maze[y-1][x] == '.':adj_cells.append([x,y-1])
        return(adj_cells)
    
    #take maze from codewars format into list of lists
    def maze_unpack(self, maze_grid):
        return([list(i) for i in  self.maze_grid.split('\n')])


class Node():
    def __init__(self, coord, maze):
        self.g = 0
        self.h = calc_manhattan_distance(coord, maze.finish)
        self.coord = coord
        self.parent = None
        self.coord_t = tuple(coord)
        
    #compute total costs from start plus estimated costs to finish
    def f(self):
        return(self.g + self.h)
    
def path_finder(maze):
    closed = set() #closed coords
    frontier = [] #heap of costs and node coordinates
    node_finder = {} #dict of coords and node objs #node_finder[coord] = Node obj
    
    maze = Maze(maze)
    start_node = Node([0,0], maze)
    
    #push a list of [costs, [x,y]]
    heappush(frontier, [start_node.f(), start_node.coord])
    
    #add coords to lookup dict to ref node object
    node_finder[start_node.coord_t] = start_node
    
    #while we have nodes...
    while frontier:
        
        #pop the cheapest node off for testing
        current_node = heappop(frontier)
        
        #if at finish coords, return our total real step cost (node.g)
        if current_node[1] == maze.finish:
            return node_finder[tuple(current_node[1])].g
        
        #add the coords to our closed/visited/seen set
        closed.add(tuple(current_node[1]))
        
        #make a list of valid neighbors to visit, only major cardinal directions with no walls
        adj_nodes = [Node(i, maze) for i in maze.get_adjacent_cells(current_node[1])]
        for node in adj_nodes:
            
            #if already tested node, skip
            if tuple(node.coord_t) in closed:
                continue            
            
            #else evaluate
            else:
                
                #add cost of move to this neighbor node to the cost of the total path here
                node.g = node_finder[tuple(current_node[1])].g + 1
                
                #check the dict to see if this total cost is less than the existing path cost to this node
                #if so, replace the node details
                if node.coord_t in node_finder:
                    if node.g < node_finder[node.coord_t].g:
                        node_finder[node.coord_t] = node
                        
                #otherwise, add neighbor nodes to the dict and to the frontier for further exploration
                else:
                    node_finder[node.coord_t] = node
                    heappush(frontier, [node.f(), node.coord])
    return(False)